# Installing and Importing and Initializing Libraries.

In [1]:
import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
tqdm.pandas()
pd.set_option('display.max_row', 500)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet


# Windowing the training data
### As the target is to detect an event, 
### So, extracting timesteps before and after the events. 

In [2]:
def window(df, win_size):
    ind=df.index[df['event'].isna()==False]
    c=0
    df['window']=np.nan
    for i in tqdm(ind):
        a=i-win_size
        b=i+win_size
        df['window'].loc[a:i]=int(c)
        c=c+1
        df['window'].loc[i:b]=int(c)
        c=c+1
    df['window'].dropna(inplace=True)
    return df[df['window'].isna()==False]

# Removing the steps where change in anglez is insignificant and enmo is zero (device is supposed not to be worn)

In [3]:
def inactive_periods(df):
    print("shape before application: ",df.shape)
    df['diff_anglez']=df['anglez'].diff()
    df=df[(df['enmo']!=0.0) | (df['diff_anglez']!=0.0)]
    print("shape after application: ",df.shape)
    df.drop('diff_anglez', inplace=True, axis=1)
    print("shape after completion: ",df.shape)
    print("removed ")
    return df


# Clustering the Enmo and Anglez

In [4]:
def clustering(df):
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import StandardScaler    
    X=df[['anglez','enmo']]
#Scalling the data
    scaler=StandardScaler()
    X_scaled=scaler.fit_transform(X)
#perform clustering
    model=KMeans(n_clusters=4,algorithm="elkan" )
    model.fit(X_scaled)
    return model.labels_


# Adding Rolling Standard Deviations

- As per observation enmo and anglez varies frequently and more than normal for few timesteps before and after the event.
- In order to consider the variations into the modelling, using the method.

In [5]:
 def rollingstd(series_df):
# Creating columns with nans
    series_df['sd_enmo_1']=np.nan    # 1 min rolling std: enmo
    series_df['sd_anglez_1']=np.nan  # 1 min rolling std: anglez
    series_df['m_enmo_2']=np.nan     # 2 min rolling mean: enmo
    series_df['m_anglez_2']=np.nan   # 2 min rolling std: anglez 
    print('anglez rolling std 12')
    series_df['sd_anglez_1'] = (series_df.groupby('series_id')['anglez']
                      .rolling(12)
                      .std()
                      .reset_index(level=0, drop=True))
    print('anglez rolling std 2')
    series_df['sd_anglez_1'][series_df['sd_anglez_1'].isna()==True] = (series_df.groupby('series_id')['anglez']
                      .rolling(2)
                      .std()
                      .reset_index(level=0, drop=True))
    print('enmo rolling std 12')
    series_df['sd_enmo_1'] = (series_df.groupby('series_id')['enmo']
                      .rolling(12)
                      .std()
                      .reset_index(level=0, drop=True))
    print('enmo rolling mean 24')
    series_df['m_enmo_2'] = (series_df.groupby('series_id')['enmo']
                      .rolling(24)
                      .mean()
                      .reset_index(level=0, drop=True))
    print('anglez rolling mean 24')
    series_df['m_anglez_2'] = (series_df.groupby('series_id')['anglez']
                      .rolling(24)
                      .mean()
                      .reset_index(level=0, drop=True))
    print('enmo rolling std 2')
    print('Nans in sd_emno_1: ',series_df['sd_enmo_1'].isnull().sum())
    series_df['sd_enmo_1'][series_df['sd_enmo_1'].isna()==True] = (series_df.groupby('series_id')['enmo']
                      .rolling(2)
                      .std()
                      .reset_index(level=0, drop=True))
    print('enmo rolling mean 2')
    series_df['m_enmo_2'][series_df['m_enmo_2'].isna()==True] = (series_df.groupby('series_id')['enmo']
                      .rolling(2)
                      .mean()
                      .reset_index(level=0, drop=True))
    print('anglez rolling mean 2')
    series_df['m_anglez_2'][series_df['m_anglez_2'].isna()==True] = (series_df.groupby('series_id')['anglez']
                      .rolling(2)
                      .mean()
                      .reset_index(level=0, drop=True))
#Series wise rolling std and mean
# filling rest of nans
    print('Nans in sd_emno_1: ',series_df['sd_enmo_1'].isnull().sum())
    series_df['sd_enmo_1'].fillna(0.0, inplace=True)
    series_df['sd_anglez_1'].fillna(0.0, inplace=True)
    series_df['m_enmo_2'].fillna(0.0, inplace=True)
    series_df['m_anglez_2'].fillna(0.0, inplace=True)
    print('Nans after removal: ',series_df['sd_enmo_1'].isnull().sum())

    return(series_df)


# Scaling the data

In [6]:
def scale(X):
    from sklearn import preprocessing
    scaler=preprocessing.StandardScaler().fit(X)
    return (scaler.transform(X))


# Creating Training Data

In [7]:
# Importing the datasets
print('Importing Training Datasets')
df_series=pd.read_parquet(path="/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet", engine='auto')
df_events=pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
print('Dataset Imported...')
print('______________________________________')

# Merging the datasets
print('Merging the training datasets...')
events=df_events[['series_id', 'step','event']]
series_df=pd.merge(df_series,events,on=["step","series_id"],how='left')
df_series=[]
df_event=[]
series_df['sleep']=np.nan
series_df.loc[series_df["event"]=="onset", "sleep"] = 1
series_df.loc[series_df["event"]=="wakeup", "sleep"] = 0
series_df['sleep'].fillna(method='ffill', inplace=True)
series_df['sleep'].fillna(value=0, inplace=True)
print('Datasets Merged...')
print('______________________________________')

# Removing the periods of inactivity
print('Removing the periods of Inactivity...')
series_df=inactive_periods(series_df)
print('______________________________________')

# Forming Windows
win_size=720  #60mins
print('Creating Windows each size: ',win_size)
series_df=window(series_df,win_size)
print('Windows formed...')
print('______________________________________')

# Adding the columns of Standard Deviation (1 min)
print('Adding columns to account for deviation in enmo and anglez 1 min rolling...')
series_df=rollingstd(series_df)
series_df['sd_anglez_1']=pd.to_numeric(series_df['sd_anglez_1'])
series_df['sd_enmo_1']=pd.to_numeric(series_df['sd_enmo_1'])
series_df['m_anglez_2']=pd.to_numeric(series_df['m_anglez_2'])
series_df['m_enmo_2']=pd.to_numeric(series_df['m_enmo_2'])
print('Std columns added...')
print('______________________________________')

# Clustering the Data
print('Clustering the data based on enmo and anglez...')
series_df['cluster']=(clustering(series_df)+1)/4
print('Added clusters...')

# Creating dataframes for training
X=series_df[['sd_anglez_1','sd_enmo_1','anglez','m_anglez_2','m_enmo_2','enmo','cluster']]
y=series_df[['sleep']]
X=scale(X)

Importing Training Datasets
Dataset Imported...
______________________________________
Merging the training datasets...
Datasets Merged...
______________________________________
Removing the periods of Inactivity...
shape before application:  (127946340, 7)
shape after application:  (111766109, 8)
shape after completion:  (111766109, 7)
removed 
______________________________________
Creating Windows each size:  720


100%|██████████| 6928/6928 [00:06<00:00, 1099.85it/s]


Windows formed...
______________________________________
Adding columns to account for deviation in enmo and anglez 1 min rolling...
anglez rolling std 12
anglez rolling std 2
enmo rolling std 12
enmo rolling mean 24
anglez rolling mean 24
enmo rolling std 2
Nans in sd_emno_1:  2959
enmo rolling mean 2
anglez rolling mean 2
Nans in sd_emno_1:  269
Nans after removal:  0
Std columns added...
______________________________________
Clustering the data based on enmo and anglez...
Added clusters...


In [8]:
# Creating dataframes for training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
import plotly.express as px
figure= px.imshow(series_df[['sd_anglez_1','sd_enmo_1','anglez','m_anglez_2','m_enmo_2','enmo','cluster','sleep']].corr(), text_auto=True, width=1200, height=1200)
figure.show()

In [20]:
from sklearn.ensemble import RandomForestClassifier
rf =RandomForestClassifier(n_jobs=-1,verbose=1) 
print ('Training the model')
rf.fit(X_train,y_train)
# clearing the RAM
X=[]
series_df=[]
y=[]

Training the model


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 14.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 33.4min finished


In [21]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import r2_score
y_pred = rf.predict(scale(X_test))
mae = mean_absolute_error(y_pred,y_test)

# R_square
r2 = r2_score(y_test, y_pred)

print('The MAE or J(theta) is: ',mae)
print('r2_score =',r2)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   25.3s


The MAE or J(theta) is:  0.09827232031121921
r2_score = 0.6011262595383713


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   58.0s finished


In [22]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_pred,y_test)

# R_square
r2 = r2_score(y_test, y_pred)

print('The MAE or J(theta) is: ',mae)
print('r2_score =',r2)

The MAE or J(theta) is:  0.09827232031121921
r2_score = 0.6011262595383713


In [23]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [24]:
params = {
    # Parameters that we are going to tune.
    'max_depth':16,
    'min_child_weight': 12,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

params['eval_metric'] = "mae"
num_boost_round = 999

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[0]	Test-mae:0.40291
[1]	Test-mae:0.33956
[2]	Test-mae:0.29477
[3]	Test-mae:0.26304
[4]	Test-mae:0.24026
[5]	Test-mae:0.22395
[6]	Test-mae:0.21217
[7]	Test-mae:0.20331
[8]	Test-mae:0.19708
[9]	Test-mae:0.19220
[10]	Test-mae:0.18862
[11]	Test-mae:0.18573
[12]	Test-mae:0.18373
[13]	Test-mae:0.18196
[14]	Test-mae:0.18056
[15]	Test-mae:0.17902
[16]	Test-mae:0.17807
[17]	Test-mae:0.17728
[18]	Test-mae:0.17639
[19]	Test-mae:0.17577
[20]	Test-mae:0.17520
[21]	Test-mae:0.17461
[22]	Test-mae:0.17424
[23]	Test-mae:0.17386
[24]	Test-mae:0.17329
[25]	Test-mae:0.17293
[26]	Test-mae:0.17260
[27]	Test-mae:0.17201
[28]	Test-mae:0.17189
[29]	Test-mae:0.17168
[30]	Test-mae:0.17134
[31]	Test-mae:0.17124
[32]	Test-mae:0.17080
[33]	Test-mae:0.17048
[34]	Test-mae:0.17033
[35]	Test-mae:0.17014
[36]	Test-mae:0.16981
[37]	Test-mae:0.16947
[38]	Test-mae:0.16927
[39]	Test-mae:0.16914
[40]	Test-mae:0.16908
[41]	Test-mae:0.16894
[42]	Test-mae:0.16883
[43]	Test-mae:0.16872
[44]	Test-mae:0.16855
[45]	Test-mae:0.1684

In [25]:
y_pred=model.predict(dtest)
mae = mean_absolute_error(y_pred,y_test)

# R_square
r2 = r2_score(y_test, y_pred)

print('The MAE or J(theta) is: ',mae)
print('r2_score =',r2)

The MAE or J(theta) is:  0.1533112961939945
r2_score = 0.7096571288155693


# Creating Test Data

In [26]:
# Importing the datasets
print('Importing Testing Datasets')
test_series=pd.read_parquet(path="/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet", engine='auto')
print('Dataset Imported...')
print('______________________________________')

# Removing the periods of inactivity
print('Removing the periods of Inactivity...')
test_series=inactive_periods(test_series)
print('______________________________________')

# Adding Features
print('Adding Features...')
test_series=rollingstd(test_series)
test_series['sd_anglez_1']=pd.to_numeric(test_series['sd_anglez_1'])
test_series['sd_enmo_1']=pd.to_numeric(test_series['sd_enmo_1'])
test_series['m_anglez_2']=pd.to_numeric(test_series['m_anglez_2'])
test_series['m_enmo_2']=pd.to_numeric(test_series['m_enmo_2'])
print('Features added...')
print('______________________________________')

# Clustering the Data
print('Clustering the data based on enmo and anglez...')
test_series['cluster']=(clustering(test_series)+1)/4
print('Added clusters...')

X_test=test_series[['sd_anglez_1','sd_enmo_1','anglez','m_anglez_2','m_enmo_2','enmo','cluster']]
# dtest=xgb.DMatrix(scale(X_test,) label=y_test)
y_pred=rf.predict(scale(X_test))


Importing Testing Datasets
Dataset Imported...
______________________________________
Removing the periods of Inactivity...
shape before application:  (450, 5)
shape after application:  (322, 6)
shape after completion:  (322, 5)
removed 
______________________________________
Adding Features...
anglez rolling std 12
anglez rolling std 2
enmo rolling std 12
enmo rolling mean 24
anglez rolling mean 24
enmo rolling std 2
Nans in sd_emno_1:  33
enmo rolling mean 2
anglez rolling mean 2
Nans in sd_emno_1:  3
Nans after removal:  0
Features added...
______________________________________
Clustering the data based on enmo and anglez...
Added clusters...


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


# Post Processing

In [27]:
result_df=test_series[['series_id', 'step','timestamp']]
result_df['sleep']=y_pred
result_df['timestamp']=result_df[['timestamp']].progress_apply(lambda x: pd.to_datetime(x,utc=True))

100%|██████████| 1/1 [00:00<00:00, 46.69it/s]


# Creating the Submission File

In [28]:
df=result_df.copy()
df.index=df['timestamp']
mean = df.groupby([df['series_id'], df.index.floor('30min')])['sleep'].mean()  # Calculating the mean of predictions over an interval of 30 mins. 
summary=pd.merge(result_df,mean,on=["timestamp","series_id"],how='left')  # merging the means into the original data based on timestamps and series ID.
summary=summary[summary['sleep_y'].isna()==False]  # removing the Nan's of prediction mean. That'll ensure that we have a row every 30 mins.
# Creating Event Column
summary['event']=np.nan
summary.loc[summary["sleep_y"]>0.5, "event"] = 'onset'  # the mean prediction will be 1 if predicted onset for 30 mins consecutive
summary.loc[summary["sleep_y"]<0.5, "event"] = 'wakeup' # the mean prediction will be 0 if predicted wakeup for 30 mins consecutive. Any duration in between will be considered disturbance as will be less tan 30 mins.
summary=summary[summary['event'].isna()==False] # Removing the rows with no event recorded. 
summary=summary.reset_index()
summary=summary.reset_index()
summary=summary.rename(columns={'level_0': 'row_id'})
summary.index=summary['row_id']
submission=summary[['series_id','step','event','sleep_y']]  # Creating Submission
submission['sleep_y'][submission['event']=='wakeup']=1-submission["sleep_y"]
submission = submission.rename(columns={'sleep_y': 'score'})  # Renaming a column
submission.to_csv('submission.csv')  # Saving the csv file


In [29]:
###XG Boost

In [30]:
dtest=xgb.DMatrix(X_test)
y_pred=model.predict(dtest)
X_test=[]
result_df=test_series[['series_id', 'step','timestamp']]
result_df['sleep']=y_pred
result_df['timestamp']=result_df[['timestamp']].progress_apply(lambda x: pd.to_datetime(x,utc=True))
df=result_df.copy()
df.index=df['timestamp']
mean = df.groupby([df['series_id'], df.index.floor('30min')])['sleep'].mean()  # Calculating the mean of predictions over an interval of 30 mins. 
summary=pd.merge(result_df,mean,on=["timestamp","series_id"],how='left')  # merging the means into the original data based on timestamps and series ID.
summary=summary[summary['sleep_y'].isna()==False]  # removing the Nan's of prediction mean. That'll ensure that we have a row every 30 mins.
# Creating Event Column
summary['event']=np.nan
summary.loc[summary["sleep_y"]>0.5, "event"] = 'onset'  # the mean prediction will be 1 if predicted onset for 30 mins consecutive
summary.loc[summary["sleep_y"]<0.5, "event"] = 'wakeup' # the mean prediction will be 0 if predicted wakeup for 30 mins consecutive. Any duration in between will be considered disturbance as will be less tan 30 mins.
summary=summary[summary['event'].isna()==False] # Removing the rows with no event recorded. 
summary=summary.reset_index()
summary=summary.reset_index()
summary=summary.rename(columns={'level_0': 'row_id'})
summary.index=summary['row_id']
submission=summary[['series_id','step','event','sleep_y']]  # Creating Submission
submission['sleep_y'][submission['event']=='wakeup']=1-submission["sleep_y"]
submission = submission.rename(columns={'sleep_y': 'score'})  # Renaming a column
submission.to_csv('submissionxg.csv')  # Saving the csv file

100%|██████████| 1/1 [00:00<00:00, 143.02it/s]
